# Домашняя-зачетная работа

Студент(ка):

Команда:

## Постановка задачи

Используя знания, полученные на предыдущих практиках предлагается решить одну из предложенные задач, при помощи обучения нейронных сетей:
* Классификация изображений (минимум 5 различных классов);
* Регрессия (например, определять по фотографии возраст/эмоции/гендер);
* Восстановление изображений (при помощи автоэнкодера).

Это примеры задач, ноутбук заточен именно под них. **НО**, если вы хотите сделать что-то своё, что-то новое -- можете делать. Только лучше предварительно согласовать со мной.

Все функции должны быть самописные, начиная от классов для работы с датасетами, заканчивая классами моделей с описанием архитектуры и циклов обучения и верификации.

Т.е. взять и сделать:
```python
from ultralytics import YOLO
```

или

```python
backbone = timm.create_model('mobilenetv3_large_100')
````

не получится :O

Ниже по ноутбуку прописаны разделы и колличество баллов, которое можно получить за качественно выполненный раздел.

Качественно выполненный, имеется ввиду, чтобы он, во-первых, выполнял свою функцию, во-вторых, имел комментарии по ходу, где и что выполняется.

На качество, красоту и оптимизацию кода я НЕ смотрю, главное, чтобы работало и было подробно описано. Поэтому делайте, как вам удобно, и как привыкли, но пишите комментарии.

## Подключение библиотек

В этом блоке рекомендуется сразу подключить все необходимые для работы библиотеки, я укажу те, которые мы использовали на практиках, и которые, скорее всего, вам и потребуются.

Но, может быть у вас будут еще какие-то, использовать можете всё, что потребуется.

Напоминаю, что если вы работаете в Google Colab, то необходимые библиотеки устанавливаются через следующую команду в ячейке ноутбука.

Локально это тоже работает, в любом ноутбуке, но локально можно и через консоль поставить)

In [ ]:
!pip install numpy

In [ ]:
from torch.utils.data import Dataset
from PIL import Image
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import random
from torchinfo import summary
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torchvision

# TODO: ваши библиотеки


Следующая команда для того, чтобы графики нормально отображались в ячейках ноутбука

In [ ]:
%matplotlib inline

## Подготовка датасета (15 баллов)

Реализовать класс для работы с датасетом, мы делали такое на [практике №4](./Practice_4/Practice4.ipynb).

Класс должен иметь методы `__len__` и `__getitem__`.

Желательно, чтобы при инициализации класса, мы могли передавать в него какие-нибудь параметры, например: количество каналов; размер, к которому приводить изображения; путь до картинок; путь до меток классов; используемые аугментации и тд.

**!!!**

Если вы добавите различные аугментации в класс датасета, это это будут дополнительнные 5 баллов. 

Удобнее это сделать через библиотеку **transforms**, с которой мы уже работали.

А типы аугментаций можно посмотреть по [ссылке](https://pytorch.org/vision/stable/transforms.html).

In [ ]:
class WorkDataset(Dataset):
    def __init__(self):
        pass

Если вы решаете задачу классификации или регрессии, то работать лучше с **csv**-файлом, который содержит метки классов или значения для регрессии, например, для классификации он выглядит следующим образом:

In [ ]:
./fullpath/cat.jpg;0
./fullpath/dog.jpg;1
./fullpath/dino.jpg;2
./fullpath/bird.jpg;3
./fullpath/elephant.jpg;4

Сначала идет полный путь до файла, затем разделитель, в данном случае это точка с запятой, затем метка класса.

Для регрессии разметка будет такая же, но, метки лучше нормализовать от 0 до 1. Т.е. если вы хотите сделать предсказание возраста, а возраст, допустим от 0 до 80. То вы делите все значения возрастов, что у вас есть на максимальное значение, и у вас будут значения от 0 до 1. 

По идее, так можно сделать с любым датасетом, и под любую задачу, т.е. через регрессию вы можете классифицировать, например, породы кошек или собак. Просто нужно поделить на максимальную метку, которая у вас есть.

## Архитектура модели (25 баллов)

Реализовать класс для самой модели. Не беритесь за что-то сложное, можно попробовать буквально пару слоёв, а потом при необходимости добавить.

Например, как выглядят простейшие архитектуры моделей для [классификации](https://www.kaggle.com/code/basu369victor/pytorch-tutorial-the-classification) и [регрессии](https://machinelearningmastery.com/building-a-regression-model-in-pytorch/).

Разница между классификацией и регрессией в том, что при классификации вы на выходе получаете вектор логитов (т.е. вероятностей отнесения входных данных к какому-то классу), а при регрессии на выходе у вас будет всего одно число.

In [ ]:
class WorkModel(nn.Module):
    def __init__(self):
        super().__init__()
        
        pass

## Train/val циклы (20 баллов)

Реализовать методы для циклов обучения и валидации.

Т.е., бежим по батчам, получаем выход модели, считаем лосс, при обучении: делаем бэквард.

Желательно, во время процесса обучения и валидации выводить значения лосса и аккураси после каждой эпохи. Плюс, строить графики.

In [ ]:
def train():
    # TODO: train loop
    pass

In [ ]:
def val():
    # TODO: val loop
    pass

## Инициализация остальных параметров и запуск обучения (15 баллов)

Инициализируете конфиг, фиксируете сид, подбираете функцию потерь:
* для классификации -- Cross-Entropy;
* для регрессии -- MSE;
* для автоэнкодера -- MSE.

Подбираете метод для подсчета аккураси по время обучения, подбираете гиперпараметры, оптимайзер, шедулер.

Будет круто, если напишите, что и почему выбрали.

In [ ]:
# TODO: код здесь

После того, как всё инициализировали можете запускать обучение

In [ ]:
# TODO: код здесь

## Визуализация результатов (10 баллов)

После того, как получили модель с достаточным качеством, нужно показать, как она работает.

Визаулизируйте данные, которые подаёте на вход, какой выход модели. Можно даже собрать тестовый датасет и посчитать на нём метрики, кроме тех, что считали во время обучения.

In [ ]:
# TODO: код здесь

## Выводы (10 баллов)

Пишите всё, с чем столкнулись во время работы, что выбирали и почему, что получилось, а что не очень. Чем более полные будут выводы, тем больше вероятность того, что вы получите максимальный балл.